In [1]:
# from rapidfuzz import process, fuzz
# import altair
import json
import numpy
import pandas
import pathlib
import pydash
import requests
import time
import tqdm
# import unidecode
import uuid

def value_extract(row, col):

    ''' Extract dictionary values. '''
  
    return pydash.get(row[col], "value")
   
def sparql_query(query, service):
 
    ''' Send sparql request, and formulate results into a dataframe. '''

    r = requests.get(service, params={"format": "json", "query": query})
    data = pydash.get(r.json(), "results.bindings")
    data = pandas.DataFrame.from_dict(data)
    for x in data.columns:
        data[x] = data.apply(value_extract, col=x, axis=1)
 
    return data

# def normalise(row, col):

#     ''' Normalise text for matching purposes. '''

#     norm = unidecode.unidecode(str(row[col]).lower()).strip()

#     return norm

# def median_score(a_list, b_id, f):

#     ''' Find best match per against lists, return median. '''

#     test = wikidata.loc[wikidata.director_wikidata.isin([b_id])]
#     b_list = test.film_label.unique()
#     if len(a_list) < f or len(b_list) < f:
#         return 0

#     my_score = [process.extractOne(a, b_list, scorer=fuzz.WRatio)[1] for a in a_list]
#     return numpy.median(my_score)

#
# reengineer this so that uuids are only created if they do not exist, and strings are assumed to be unique.
#



def assign_uuid(entity_type, string):
    if string not in entities[entity_type].keys():
        entities[entity_type][string] = str(uuid.uuid4())

    return {entity_type: string, f'{entity_type}_id':entities[entity_type][string]}
        # print('FOUND ONE')
    # else:
    #     entities[entity_type][string] = 
    #     # return {}
    #     # print('NOPOOOOOO')
    
    # return string

    # what you do here, does it already exist in the class, if so return that, otherwise make a new


entities = {'country':{}, 'director':{}}


data_path = pathlib.Path.cwd() / 'sight_and_sound.json'

if not data_path.exists():

    index = requests.get('https://www.bfi.org.uk/sight-and-sound/greatest-films-all-time/all-voters').text
    index = index.split('<script type="text/javascript">var initialPageState = ')[1].split('</script>')[0]
    index = pydash.get(json.loads(index), 'componentState.allVoters')

    data = list()

    for x in tqdm.tqdm(index):

        time.sleep(4)

        voter = {k:v for k,v in x.items() if k in ['firstname', 'surname', 'type', 'country', 'url']}
        voter['voter_id'] = str(uuid.uuid4())
        voter['country'] = [assign_uuid('country', x) for x in voter['country'].split('/')]

        votes = pandas.read_html('https://www.bfi.org.uk'+voter['url'], encoding='utf8')[0].to_dict('records')
        for y in votes:
            y['film_id'] = str(uuid.uuid4())
            y['Director'] = [assign_uuid('director', x) for x in str(y['Director']).split(',')]

        voter['votes'] = votes
        data.append(voter)

    with open(data_path, 'w') as write_data:
        json.dump(data, write_data, ensure_ascii=False, indent=4)
else:
    with open(data_path) as read_data:
        data = json.load(read_data)

print(json.dumps(data[-1], indent=4, ensure_ascii=False))

{
    "firstname": "Pedro Adrián",
    "surname": "Zuluaga",
    "type": "critic",
    "country": [
        {
            "country": "Colombia",
            "country_id": "ce20dbd2-4c27-4728-a7d0-2b8a0c2ce0a4"
        }
    ],
    "url": "/sight-and-sound/greatest-films-all-time/all-voters/pedro-adrian-zuluaga",
    "voter_id": "86a6ed28-cca8-4d66-90dc-0b82669ee5d3",
    "votes": [
        {
            "Film": "The Virgin Spring",
            "Year": 1960,
            "Director": [
                {
                    "director": "Ingmar Bergman",
                    "director_id": "0c3b11e7-42e2-4f5b-b333-53381e2ac0ee"
                }
            ],
            "film_id": "f51b2d87-c62d-4801-be73-4f1f0d80b33d"
        },
        {
            "Film": "THÉRÈSE",
            "Year": 1986,
            "Director": [
                {
                    "director": "Alain Cavalier",
                    "director_id": "369e805c-9f61-4b91-92ac-31c8a79e52f9"
                }
           

In [2]:
# reconcile countries with wikidata.

query = '''select ?country ?countryLabel 
    where {
      values ?status {wd:Q3624078 wd:Q6256 wd:Q779415}
        ?country wdt:P31 ?status .
        service wikibase:label { bd:serviceParam wikibase:language "en". }}'''

wikidata_country = sparql_query(query, "https://query.wikidata.org/sparql")
wikidata_country = wikidata_country.rename(columns={'country':'country_wikidata', 'countryLabel':'country'})
wikidata_country['country_wikidata'] = wikidata_country['country_wikidata'].str.split('/').str[-1]

country = pandas.json_normalize(data, record_path=['country'])
country = country.replace({'country':{
    'UK':'United Kingdom', 
    'Uk':'United Kingdom', 
    'England':'United Kingdom', 
    'USA':'United States of America', 
    'US':'United States of America', 
    'United States':'United States of America', 
    'Ireland':'Republic of Ireland',
    'China':"People's Republic of China",
    'Finnland':'Finland',
    'france':'France',
    'Canda':'Canada',
    'Palestine':'State of Palestine', 
    'Macedonia':'North Macedonia',
    'Abu Dhabi':'United Arab Emirates'
    }})

country = pandas.merge(country, wikidata_country, on='country', how='left').drop_duplicates()
country.loc[country.country_wikidata.isin([numpy.nan]), 'country_wikidata'] = None
country = country[['country_id', 'country_wikidata']]
country = {a:b for a,b in zip(country.country_id, country.country_wikidata)}

for x in data:
    for y in x['country']:
        y['country_wikidata'] = country[y['country_id']]

print(json.dumps(data[-1], indent=4, ensure_ascii=False))

In [ ]:
wikidata_path = pathlib.Path.cwd() / 'wikidata.parquet'

if not wikidata_path.exists():
    wikidata = pandas.DataFrame()
    for year in tqdm.tqdm(range(1880, 2025)):
        query = '''select ?film ?filmLabel ?title ?director ?directorLabel (year(?publication_date) as ?year) 
            where {
                ?film p:P31/wdt:P279* ?state .
                ?state ps:P31/wdt:P279* wd:Q11424 .
                ?film  wdt:P577 ?publication_date .
                filter (year(?publication_date) = '''+str(year)+''') .
                ?film wdt:P57 ?director
                optional { ?film wdt:P1476 ?title } .
                service wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}'''
        extract = sparql_query(query, "https://query.wikidata.org/sparql")
        wikidata = pandas.concat([wikidata, extract])

    for x in ['film', 'director']:
        wikidata[x] = wikidata[x].str.split('/').str[-1]

    wikidata = pandas.concat([
        wikidata[[x for x in wikidata.columns.values if x != 'filmLabel']],
        wikidata[[x for x in wikidata.columns.values if x != 'title']].rename(columns={'filmLabel':'title'})
        ]).dropna().drop_duplicates()

    wikidata = wikidata.rename(columns={
        'film':'film_wikidata', 'director':'director_wikidata', 'title':'film_label', 'directorLabel':'director_label'})
    
    wikidata = wikidata.astype(str)
    wikidata.to_parquet(wikidata_path)
else:
    wikidata = pandas.read_parquet(wikidata_path)

print(len(wikidata)) 
wikidata.head()

In [ ]:
# # reconcile filmographies with wikidata.

# works = pandas.json_normalize(data, record_path=['votes'])
# print(len(works))
# works.head()

In [ ]:
# dataframe['country'] = dataframe['country'].str.split('/')
# dataframe = dataframe.explode('country')
# dataframe['country'] = dataframe.apply(normalise, col='country', axis=1)

# dataframe = dataframe.replace({'country': {
#     'uk':'united kingdom', 'england':'united kingdom', 'usa':'united states of america', 
#     'united states':'united states of america', 'us':'united states of america', 
#     'china':"people's republic of china", 'ireland':'republic of ireland',
#     'canda':'canada', 'palestine':'state of palestine', 'finnland':'finland',
#     'macedonia':'north macedonia', 'abu dhabi':'united arab emirates'}})

# query = '''select ?country ?countryLabel 
#     where {
#       values ?status {wd:Q3624078 wd:Q6256 wd:Q779415}
#         ?country wdt:P31 ?status .
#         service wikibase:label { bd:serviceParam wikibase:language "en". }}'''

# countries = sparql_query(query, "https://query.wikidata.org/sparql")
# countries = countries.rename(columns={'country':'country_wikidata'})
# countries = countries.rename(columns={'countryLabel':'country'})
# countries['country'] = countries.apply(normalise, col='country', axis=1)
# countries['country_wikidata'] = countries['country_wikidata'].str.split('/').str[-1]

# dataframe = pandas.merge(dataframe, countries, on='country', how='left').drop_duplicates()

# print(len(dataframe))
# dataframe.head()

In [ ]:
# wikidata_path = pathlib.Path.cwd() / 'wikidata.parquet'

# if not wikidata_path.exists():
#     wikidata = pandas.DataFrame()
#     for year in tqdm.tqdm(range(1880, 2025)):
#         query = '''select ?film ?filmLabel ?title ?director ?directorLabel (year(?publication_date) as ?year) 
#             where {
#                 ?film p:P31/wdt:P279* ?state .
#                 ?state ps:P31/wdt:P279* wd:Q11424 .
#                 ?film  wdt:P577 ?publication_date .
#                 filter (year(?publication_date) = '''+str(year)+''') .
#                 ?film wdt:P57 ?director
#                 optional { ?film wdt:P1476 ?title } .
#                 service wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}'''
#         extract = sparql_query(query, "https://query.wikidata.org/sparql")
#         wikidata = pandas.concat([wikidata, extract])

#     for x in ['film', 'director']:
#         wikidata[x] = wikidata[x].str.split('/').str[-1]

#     wikidata = pandas.concat([
#         wikidata[[x for x in wikidata.columns.values if x != 'filmLabel']],
#         wikidata[[x for x in wikidata.columns.values if x != 'title']].rename(columns={'filmLabel':'title'})
#         ]).dropna().drop_duplicates()

#     wikidata = wikidata.rename(columns={
#         'film':'film_wikidata', 'director':'director_wikidata', 'title':'film_label', 'directorLabel':'director_label'})
    
#     wikidata = wikidata.astype(str)
#     wikidata.to_parquet(wikidata_path)
# else:
#     wikidata = pandas.read_parquet(wikidata_path)

# wikidata['film_label'] = wikidata.apply(normalise, col='film_label', axis=1)
# wikidata['director_label'] = wikidata.apply(normalise, col='director_label', axis=1)

# print(len(wikidata)) 
# wikidata.head()

In [ ]:
# match_path = pathlib.Path.cwd() / 'match.parquet'

# if not match_path.exists():

#     name_match_score = 60 # name matching tolerance
#     title_match_score = 100 # title matching tolerance
#     minimum_match_candidates = 4 # minimum matching options.

#     result_dataframe = pandas.DataFrame(columns=['Director', 'director_wikidata'])
#     for x in tqdm.tqdm(dataframe.Director.unique()):
#         focus = dataframe.loc[dataframe.Director.isin([x])]
#         c = process.extract(x, wikidata.director_label.unique(), scorer=fuzz.WRatio, limit=200)
#         c = [y[0] for y in c if y[1] > name_match_score]
#         candidates = wikidata.loc[wikidata.director_label.isin(c)] 
#         result = {y:median_score(focus.Film.unique(), y, minimum_match_candidates) for y in candidates.director_wikidata.unique()}
#         result = [k for k,v in result.items() if v == title_match_score]
#         if len(result) == 1:
#             result_dataframe.loc[len(result_dataframe)] = [(x), (result[0])]
 
#     result_dataframe = result_dataframe.astype(str)
#     result_dataframe.to_parquet(match_path)
# else:
#     result_dataframe = pandas.read_parquet(match_path)

# print(len(result_dataframe))
# result_dataframe.head()

In [ ]:
# wd = wikidata.copy()
# wd = wd[['director_wikidata', 'film_label', 'film_wikidata']]
# wd = wd.rename(columns={'film_label':'Film'}).drop_duplicates()

# # result = dataframe.copy()
# dataframe = pandas.merge(dataframe, result_dataframe, on='Director', how='left')
# dataframe = pandas.merge(dataframe, wd, on=['director_wikidata', 'Film'], how='left')

# print(len(dataframe))
# dataframe.head(10)

In [ ]:
# country_dataframe = pandas.DataFrame()
# for x in tqdm.tqdm(numpy.array_split(dataframe.dropna().film_wikidata.unique(), 10)):
#     time.sleep(2)
#     film_values = ' '.join([f'wd:{y}' for y in x])
#     query = '''select ?film_wikidata ?wikidata_country 
#         where {
#             values ?film_wikidata {'''+film_values+'''}
#             ?film_wikidata wdt:P495 ?wikidata_country .}'''
#     country_dataframe = pandas.concat([country_dataframe, sparql_query(query, "https://query.wikidata.org/sparql")])

# for x in ['film_wikidata', 'wikidata_country']:
#     country_dataframe[x] = country_dataframe[x].str.split('/').str[-1]

# dataframe = pandas.merge(dataframe, country_dataframe, on='film_wikidata', how='left')
# dataframe.loc[dataframe.country_wikidata == dataframe.wikidata_country, 'match'] = 'true'
# dataframe.loc[~dataframe.match.isin(['true']), 'match'] = 'false'
# dataframe = dataframe.sort_values(by='match', ascending=False)
# dataframe = dataframe[['firstname', 'surname', 'film_wikidata', 'country',  'match']].dropna()
# dataframe = dataframe.drop_duplicates(subset=['firstname', 'surname', 'film_wikidata', 'country'], keep='first')
# dataframe = dataframe.loc[~dataframe.country.isin([''])]
# dataframe['country'] = dataframe['country'].str.title()
# dataframe['match'] = dataframe['match'].str.title()

# print(len(dataframe)) 
# dataframe.head()

In [ ]:
# altair.data_transformers.enable('default', max_rows=None)
# altair.Chart(dataframe).mark_bar().encode(
#     x = altair.X('country', title='Country of voter.'),
#     y=altair.Y('count(match)', stack="normalize", title='Cinema of own country.'),
#     color=altair.Color('match', scale=altair.Scale(domain=['True', 'False'], range=['#653E59', '#C9A6BE']),  
#                        sort=['true', 'false'], title='')).properties(width=1500, height=300, title='Sight & Sound 2022 - Patriotic Voters')